# Specifying activity models

Activity models are needed to accurately represent the non-ideal thermodynamic behavior of aqueous, liquid, gaseous, solid, and plasma phases. They are required when computing thermodynamic properties of phases and their species, such as molar enthalpies, molar heat capacities, molar volume, molar entropies, molar internal energies, species activities, species chemical potentials, etc.. These properties can only be accurate if proper activity models are assigned to the phases. For example, for an aqueous solution at extreme saline conditions, an activity model designed for low salinity conditions will not perform well.

In Reaktoro, an activity model assigned to a phase $\pi$ is a function for which the following inputs are given:

* temperature (in K), $T$
* pressure (in Pa), $P$
* mole fractions of the species in phase $\pi$, $x=(x_1,\ldots,x_\mathrm{N_{\pi}})$

The evaluation of this activity model produces the following *primary excess/residual thermodynamic properties* for the corresponding phase and its species:

* the excess/residual molar Gibbs energy of the phase (in units of J/mol), $G_\mathrm{ex}$

* the excess/residual molar enthalpy of the phase (in units of J/mol), $H_\mathrm{ex}$

* the excess/residual molar volume of the phase (in m<sup>3</sup>/mol), $V_\mathrm{ex}$

* the temperature derivative of the excess/residual molar volume at constant pressure (in m<sup>3</sup>/(mol·K)), $(\partial V_\mathrm{ex}/\partial T)_P$

* the pressure derivative of the excess/residual molar volume at constant temperature (in m<sup>3</sup>/(mol·Pa)), $(\partial V_\mathrm{ex}/\partial P)_T$

* the excess/residual molar isobaric heat capacity of the phase (in units of J/(mol·K)), $C_{P,\mathrm{ex}}$

* the activity coefficient $\gamma_i$ of every species in the phase, for $i=1,\ldots,\mathrm{N_{\pi}}$

* the activity $a_i$ of every species in the phase, for $i=1,\ldots,\mathrm{N_{\pi}}$

Once we have computed these *primary excess/residual thermodynamic properties*, we can determine all others if needed (i.e., the *secondary excess/residual thermodynamic properties*). For example, the excess internal energy can be computed as $U_\mathrm{ex} = H_\mathrm{ex} - P V_\mathrm{ex}$. 

By default, all phases in Reaktoro are created with *ideal activity models* if not explicitly specified. The ideal models produce zero excess/residual properties for the phase and unit activity coefficients (thus, no correction for the non-ideal thermodynamic behavior of phases). 

The code block that we present below:

In [18]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C Ca Mg"))
gases = GaseousPhase("CO2(g) H2O(g)")
solidsolution = MineralPhase("Siderite Rhodochrosite")
ionexchange = IonExchangePhase("NaX CaX2")

system = ChemicalSystem(db, solution, gases, solidsolution, ionexchange)

is thus equivalent to this one:

In [19]:
from reaktoro import *

db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase(speciate("H O Na Cl C Ca Mg"))
solution.setActivityModel(ActivityModelIdealAqueous())

gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelIdealGas())

solidsolution = MineralPhase("Siderite Rhodochrosite")
solidsolution.setActivityModel(ActivityModelIdealSolution(StateOfMatter.Solid))

ionexchange = IonExchangePhase("NaX CaX2")
ionexchange.setActivityModel(ActivityModelIdealIonExchange())

system = ChemicalSystem(db, solution, gases, solidsolution, ionexchange)

Non-ideal thermodynamic behavior is often what most phases present in most conditions. Therefore, it is imperative to properly select an activity model for a phase suitable for the conditions of interest. Because there is no model that works best for all conditions, we leave the choice for the user.

We show in the following sections the available options for activity models for different phases and how to set and configure them.

## Specifying activity models for aqueous phases

The following are additional activity models available for aqueous phases in Reaktoro:

* `ActivityModelDebyeHuckel`
* `ActivityModelDebyeHuckelKielland`
* `ActivityModelDebyeHuckelLimitingLaw`
* `ActivityModelDebyeHuckelParams`
* `ActivityModelDebyeHuckelPHREEQC`
* `ActivityModelDebyeHuckelWATEQ4F`
* `ActivityModelHKF`
* `ActivityModelPitzerHMW`

For example, the code block below will construct a chemical system with an aqueous phase that uses the HKF activity model {cite}`Helgeson1981`, which is fairly accurate for solutions up to 6 molal NaCl.

In [20]:
solution = AqueousPhase(speciate("H O Na Cl"))
solution.setActivityModel(ActivityModelHKF())

system = ChemicalSystem(db, solution)

Let's create a chemical state for this system (using {{ChemicalState}} class), and set it so that we have a 1 molal NaCl saline solution. We then compute its chemical properties (using class {{ChemicalProps}}) and print the activity coefficients, $\gamma_i$, of the species.

In [21]:
state = ChemicalState(system)
state.temperature(25, "celsius")
state.pressure(1, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")

print(f"{'Species':<20}{'Activity Coefficient'}")
props = ChemicalProps(state)
for species in system.species():
    print(f"{species.name():<20}{props.speciesActivityCoefficient(species.name())}")

Species             Activity Coefficient
H+                  0.614425
H2O                 1.00142
Cl-                 0.665158
H2                  1.25894
Na+                 0.65199
OH-                 0.718866
NaOH                1.25894
O2                  1.25894


Note above that the activity coefficients for neutral species other than H{{_2}}O are identical. This is because most aqueous activity models do not properly compute activity coefficients for these species (or their default variants do not contain model parameters for them). 

One way to overcome or improve this is to combine an aqueous activity model with *Setschenow model*, so that the following equation:

$$\log_{10}\gamma_i = b_i I$$

is used instead to compute the activity coefficient of a neutral species $i$, where $I$ is the ionic strength of the solution and $b_i$ is a *Setschenow constant*. 

We recreate the chemical system in the next code block in which a *chained activity model* is constructed so that the above Setschenow equation is applied for the activity coefficients of species `O2`, `H2`, and `NaOH`:

In [22]:
solution = AqueousPhase(speciate("H O Na Cl"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelSetschenow("O2", 0.123),
    ActivityModelSetschenow("H2", 0.234),
    ActivityModelSetschenow("NaOH", 0.345),
))

system = ChemicalSystem(db, solution)

We can then repeat the process of computing the chemical properties of the new system and printing the activity coefficients of the species:

In [23]:
state = ChemicalState(system)
state.temperature(25, "celsius")
state.pressure(1, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")

print(f"{'Species':<20}{'Activity Coefficient'}")
props = ChemicalProps(state)
for species in system.species():
    print(f"{species.name():<20}{props.speciesActivityCoefficient(species.name())}")

Species             Activity Coefficient
H+                  0.614425
H2O                 1.00142
Cl-                 0.665158
H2                  1.71399
Na+                 0.65199
OH-                 0.718866
NaOH                2.21317
O2                  1.32741


Note that the activity coefficients of `O2`, `H2`, and `NaOH` are now different from each other. No changes were applied to the activity coefficients of `H2O` and charged species.

CO<sub>2</sub> is a common gas dissolved in aqueous solutions. Most of the activity models above also do not properly calculate the activity of this aqueous species (with exception of `ActivityModelPitzerHMW`). Thus, when using such models, it is recommended to create a *chained activity model* in which one of the following models below are combined with one of the above:

* `ActivityModelDrummond`
* `ActivityModelDuanSun`
* `ActivityModelRumpf`

Here is an example:

In [24]:
solution = AqueousPhase(speciate("H O Na Cl C Ca Mg"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

system = ChemicalSystem(db, solution)

With the change above, the Drummond model {cite}`drummond1981boiling` will be used for the computation of the activity of CO<sub>2</sub> aqueous species (which can be named differently across the databases supported by Reaktoro, such as `CO2` in PHREEQC databases, `CO2(aq)` in SUPCRT/SUPCRTB databases, or `CO2@` in ThermoFun databases). 

```{tip}
When computing gas solubilities in aqueous saline solutions, it is important that the activity of the dissolved gas species (e.g., `CO2(aq)`) is adequately computed.
```

## Specifying activity models for gaseous and liquid phases

Reaktoro implements a general form of the cubic equation of state as presented in {cite:t}`Smith2005` from which classic models are derived. You can assign one of the following models to an {{GaseousPhase}} or {{LiquidPhase}} object:

* `ActivityModelPengRobinson` ({cite:t}`Peng1976`)
* `ActivityModelRedlichKwong` ({cite:t}`Redlich1949`)
* `ActivityModelSoaveRedlichKwong`  ({cite:t}`Soave1972`)
* `ActivityModelVanDerWaals`  ({cite:t}`VanderWaals1873`)

Reaktoro also implements some equations of state designed for specific gaseous phases:

* `ActivityModelSpycherPruessEnnis` (for H{{_2}}O-CO{{_2}} gas mixtures, {cite:t}`Spycher2003`)
* `ActivityModelSpycherReed` (for H{{_2}}O-CO{{_2}}-CH{{_4}} gas mixtures, {cite:t}`Spycher1988`)

The example below demonstrates the use of {cite:t}`Peng1976` equation of state as the activity model for a gaseous phase:

In [25]:
gases = GaseousPhase("CO2(g) CH4(g) H2O(g) O2(g) H2(g)")
gases.setActivityModel(ActivityModelPengRobinson())

system = ChemicalSystem(db, gases)

Let's create a chemical state for this system and compute its thermodynamic and chemical properties, from which we will obtain the fugacity coefficients of the gases:

In [26]:
state = ChemicalState(system)
state.temperature(100.0, "celsius")
state.pressure(1.0, "MPa")
state.set("CO2(g)", 0.80, "mol")
state.set("CH4(g)", 0.10, "mol")
state.set("H2O(g)", 0.05, "mol")
state.set("O2(g)",  0.03, "mol")
state.set("H2(g)",  0.02, "mol")

props = ChemicalProps(state)

print(f"{'Gas':<10}{'Fugacity Coefficient'}")
for i in range(system.species().size()):
    print(f"{system.species(i).name():<10}{props.speciesActivityCoefficient(i)}")

Gas       Fugacity Coefficient
CO2(g)    0.973811
CH4(g)    0.992728
H2O(g)    0.930723
O2(g)     1.00488
H2(g)     1.02158


```{note}
Note above that the method `ChemicalProps.speciesActivityCoefficient` is used for retrieving the fugacity coefficients of the gases. The *activity coefficient of a gas is identical to its fugacity coefficient*, and the *fugacity of a gas has identical value to its activity* when reference pressure is 1 bar, which is the case adopted in Reaktoro.
```

We print below the full computed properties of the system (in this case containing only a gaseous phase):

In [27]:
print(props)

+----------------------------------------+-------------+-----------+
| Property                               |       Value |      Unit |
+----------------------------------------+-------------+-----------+
| Temperature                            |      373.15 |         K |
| Pressure                               |       1e+06 |        Pa |
| Volume                                 |  0.00302506 |        m3 |
| Gibbs Energy                           |     -117619 |         J |
| Enthalpy                               |     16254.9 |         J |
| Entropy                                |     358.768 |       J/K |
| Internal Energy                        |     13229.8 |         J |
| Helmholtz Energy                       |     -120645 |         J |
| Charge                                 |           0 |       mol |
| Element Amount:                        |             |           |
| :: H                                   |        0.54 |       mol |
| :: C                            

## Specifying activity models for pure mineral and condensed phases

Pure minerals and condensed phases (substances in solid or liquid states) constitute phases with a single species. For these pure phases, there is no need to specify an activity model other than the ideal. For solid solution phases, check the next section.

## Specifying activity models for solid solution phases

Reaktoro supports currently only the following activity models for solid solutions:

* `ActivityModelRedlichKister`

The Redlich-Kister model assumes a solution with two solid species, and their activity coefficients, $\gamma_1$ and $\gamma_2$, are calculated as follows:

$$\ln\gamma_{1}=x_{2}^{2}[a_{0}+a_{1}(3x_{1}-x_{2})+a_{2}(x_{1}-x_{2})(5x_{1}-x_{2}),$$
$$\ln\gamma_{2}=x_{1}^{2}[a_{0}-a_{1}(3x_{2}-x_{1})+a_{2}(x_{2}-x_{1})(5x_{2}-x_{1}),$$

where $a_0$, $a_1$, $a_2$ are model parameters; $x_i$ is the mole fraction of the species $i$ in the solution.

We show below how to create a chemical system with a single solid solution phase formed with minerals K-feldspar (KAlSi{{_3}}O{{_8}}) and albite (NaAlSi{{_3}}O{{_8}}) with assiged Redlich-Kister activity model:

In [28]:
a0, a1, a2 = 1.0, 2.0, 3.0  # the Redlich-Kister parameters (demonstration values!)

solidsolution = SolidPhase("K-feldspar Albite")
solidsolution.setActivityModel(ActivityModelRedlichKister(a0, a1, a2))

system = ChemicalSystem(db, solidsolution)

The following code block creates a chemical state for this system with a single solid solution phase and prints its chemical properties.

In [29]:
state = ChemicalState(system)
state.set("K-feldspar", 0.5, "mol")
state.set("Albite", 0.5, "mol")

props = ChemicalProps(state)
print(props)

+----------------------------------------+------------+-----------+
| Property                               |      Value |      Unit |
+----------------------------------------+------------+-----------+
| Temperature                            |     298.15 |         K |
| Pressure                               |     100000 |        Pa |
| Volume                                 | 0.00010473 |        m3 |
| Gibbs Energy                           |    19899.3 |         J |
| Enthalpy                               |      58946 |         J |
| Entropy                                |    130.963 |       J/K |
| Internal Energy                        |    58935.5 |         J |
| Helmholtz Energy                       |    19888.8 |         J |
| Charge                                 |          0 |       mol |
| Element Amount:                        |            |           |
| :: O                                   |          8 |       mol |
| :: Na                                  |      

## Specifying activity models for ion exchange phases

By default, ion exchange phases are created in Reaktoro with an ideal activity model assigned to it. This ideal model computes the activity of the $i$-th exchange species (e.g., NaX, KX, CaX{{_2}}) according to:

$$a_i = \dfrac{x_i z_{\mathrm{e},i}}{\sum_k x_k z_{\mathrm{e},k} } $$

where $z_{\mathrm{e},i}$ is the *exchanger's equivalent* (e.g., 1 for NaX, 2 for CaX{{_2}}) and $x_i$ is the mole fraction of the exchange species.

For non-ideal activity models for ion exchange phases, the following is currently available in Reaktoro:

* `ActivityModelIonExchangeGainesThomas`

The Gaines-Thomas model is implemented according to what is used PHREEQC {cite}`Parkhurst2013`. We show below a demonstration in which a chemical system with aqueous and ion exchange phases are created followed by the computation of the system's chemical properties at a given chemical state:

In [30]:
db = PhreeqcDatabase("phreeqc.dat")

solution = AqueousPhase("H2O Na+ Cl- H+ OH- K+ Ca+2 Mg+2")
solution.setActivityModel(ActivityModelDebyeHuckel())

exchange = IonExchangePhase("NaX KX CaX2 MgX2")
exchange.setActivityModel(ActivityModelIonExchangeGainesThomas())

system = ChemicalSystem(db, solution, exchange)

state = ChemicalState(system)
state.set("H2O" , 1.00, "kg")
state.set("Na+" , 1.00, "mmol")
state.set("K+"  , 1.00, "mmol")
state.set("Mg+2", 1.00, "mmol")
state.set("Ca+2", 1.00, "mmol")
state.set("NaX" , 0.06, "umol")
state.set("KX" ,  0.02, "umol")
state.set("CaX2" ,0.01, "umol")
state.set("MgX2" ,0.01, "umol")

props = ChemicalProps(state)
print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |       298.15 |         K |
| Pressure                               |       100000 |        Pa |
| Volume                                 |   0.00100293 |        m3 |
| Gibbs Energy                           | -0.000159825 |         J |
| Enthalpy                               |  5.99996e-05 |         J |
| Entropy                                |  7.37294e-07 |       J/K |
| Internal Energy                        |     -100.293 |         J |
| Helmholtz Energy                       |     -100.294 |         J |
| Charge                                 |        0.006 |       mol |
| Element Amount:                        |              |           |
| :: X                                   |      1.2e-07 |       mol |
| :: H              

## Recap

This guide demonstrated how you can create chemical systems with activity models of your choice assigned to the phases. Reaktoro is constantly under development and new activity models will be available over time. If you would like to help with the development of a specific activity model, [please get in touch](mailto:allan.leal@erdw.ethz.ch).

```{admonition} Attention
Ensure you have correctly selected and attached an activity model to a phase when comparing Reaktoro's computations with other codes. When benchmarking, it is important to choose similar models. Ideally, you want to select the same one and use identical parameters for the model.
```